In [28]:
# If kernel is not detected: F1 -> Developer: Reload Window

In [40]:
from langchain.llms import OpenAIChat
from langchain import LLMChain
from langchain.text_splitter import CharacterTextSplitter,TokenTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts.few_shot import FewShotPromptTemplate


import os
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
pd.set_option('display.max_colwidth', None)
from tqdm.notebook import tqdm
tqdm.pandas()
from ast import literal_eval
import random
import re
from langchain.chat_models import ChatOpenAI


In [41]:
!source ../../../elbaff_iesta_venv/bin/activate
# %pip install langchain
# %pip install python-dotenv
#%pip install openai

In [42]:
"""

liberal_chat_prompt = ChatPromptTemplate.from_messages(create_prompt_template(prompt_dict["all"].format(ideology ="liberal")))
llm_chain = LLMChain(llm=chat, prompt=liberal_chat_prompt)
result = llm_chain.run(ineffective_argument="If there was no Kryptonite, can Superman defeat the Silver Surfer?")
print(result)
cons_chat_prompt = ChatPromptTemplate.from_messages(create_prompt_template(prompt_dict["all"].format(ideology ="conservative")))
llm_chain = LLMChain(llm=chat, prompt=cons_chat_prompt)
result = llm_chain.run(ineffective_argument="If there was no Kryptonite, can Superman defeat the Silver Surfer?")
print(result)
"""

'\n\nliberal_chat_prompt = ChatPromptTemplate.from_messages(create_prompt_template(prompt_dict["all"].format(ideology ="liberal")))\nllm_chain = LLMChain(llm=chat, prompt=liberal_chat_prompt)\nresult = llm_chain.run(ineffective_argument="If there was no Kryptonite, can Superman defeat the Silver Surfer?")\nprint(result)\ncons_chat_prompt = ChatPromptTemplate.from_messages(create_prompt_template(prompt_dict["all"].format(ideology ="conservative")))\nllm_chain = LLMChain(llm=chat, prompt=cons_chat_prompt)\nresult = llm_chain.run(ineffective_argument="If there was no Kryptonite, can Superman defeat the Silver Surfer?")\nprint(result)\n'

In [43]:
# Create a new OpenAI instance


In [44]:

def create_prompt_template(prompt):
    system_message_prompt = SystemMessagePromptTemplate.from_template(prompt)
    human_template="{ineffective_argument}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    return [system_message_prompt, human_message_prompt]

In [45]:

from datasets import load_dataset, Dataset

from iesta.machine_learning.huggingface_loader import IESTAHuggingFace

def get_data(ideology, effect='ineffective', limit=500):
    name:str = f'notaphoenix/debateorg_w_effect_for_{ideology}'
    dataset: Dataset = load_dataset(name, split="test")
    dataset = dataset.filter(lambda x: x["label"] == IESTAHuggingFace._LABEL2ID_[effect]).shuffle(seed=2062021)
    
    if len(dataset) > limit:
        dataset = dataset.select(range(limit))
    print(f"Return dataset {name} with {len(dataset)} ")
    dataset = dataset.map(lambda example, idx: {"id": idx, **example}, with_indices=True)

    return dataset

    



In [37]:
from dotenv import load_dotenv, find_dotenv
found = load_dotenv(find_dotenv())

from langchain import HuggingFaceHub




def get_model(model_name):
    if model_name == "chatgpt":
        return ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    elif model_name == "falcon":
        return HuggingFaceHub(huggingfacehub_api_token=os.getenv('HUGGINGFACE_TOKEN'),
                              repo_id="tiiuae/falcon-7b-instruct", 
                              model_kwargs={"temperature":0, "max_new_tokens":1000})

chat = get_model("falcon")

In [38]:


def get_generations(ineffective_argument:str, ideology:str, llm:str="chatgpt"):
    
    result_dict = {}
    for k, prompt_template in prompt_dict.items():
        chat_prompt = ChatPromptTemplate.from_messages(create_prompt_template(prompt_template.format(ideology=ideology)))
        llm_chain = LLMChain(llm=chat, prompt=chat_prompt)
        result_dict[k] = llm_chain.run(ineffective_argument=ineffective_argument)
        #print(f"'{result}'\n\n")
    return result_dict



    


In [ ]:
import json
import pandas as pd
from tqdm import tqdm
from os.path import exists

def generate_args(ideology:str, out_file :str = "../data/llms_out/") -> pd.DataFrame:
    out_file = f"{out_file}{ideology}_gpt3.5turbo.jsonl"
    
    existing_indices = []
    
    if exists(out_file):
        _df = pd.read_json(path_or_buf=out_file, lines=True)
        existing_indices = _df['id'].values.tolist()
    
    filtered_dataset = get_data(ideology, effect="ineffective", limit=500)
    add_new_l = False
    if len(existing_indices) > 0 :
        print(f"filtering out existing indices ({len(existing_indices)})")
        filtered_dataset = filtered_dataset.filter(lambda example: example['id'] not in existing_indices)
        print(f"{filtered_dataset.num_rows} to go...")
        add_new_l = True
    
    with open(out_file, 'a') as file:

        for datapoint in tqdm(filtered_dataset):
            try:    
                
                promt_generated_dict = get_generations(datapoint["text"], ideology)
                promt_generated_dict.update(datapoint)

                nline = "\n" if add_new_l else ""

                file.write(f"{nline}{json.dumps(promt_generated_dict)}")
                add_new_l = True
            except Exception as e:

                print(e)
                print(f"Failed to get a response for ID: {datapoint['id']}")   

    

        

In [ ]:
generate_args(ideology="liberal")

In [ ]:
generate_args(ideology="conservative")

In [ ]:
load_dataset("notaphoenix/debateorg_w_effect_for_liberal", split="test")[0]

In [ ]:
for k, v in result_dict.items():
    print(f"\n{k} - {prompt_dict[k]}")
    diff = Redlines(ineffective_argument,v)
    display(Markdown(diff.output_markdown))

In [ ]:
import itertools
for ptype1, ptype2 in itertools.combinations_with_replacement(prompt_dict.keys(), 2):
    if ptype1 == ptype2:
        continue
    print(f"\n{ptype1} VS. {ptype2}")
    print(len(f"{ptype1} VS. {ptype2}")*"-")

    diff = Redlines(result_dict[ptype1],result_dict[ptype2])
    display(Markdown(diff.output_markdown))

In [ ]:
for k, v in result_dict.items():
    for k,v  in result_dict.items():
    print(f"\n{k} - {prompt_dict[k]}")
    diff = Redlines(ineffective_argument,v)
    display(Markdown(diff.output_markdown))